<a href="https://colab.research.google.com/github/pabrey/datasciencecoursera/blob/master/4b_all_routes_mean_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previous

In [5]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
import json
from google.colab import files
import time
from datetime import datetime
import math

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create example with data

## load data

In [7]:
Location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_mean_00.csv'
Separator = ';'

df =  pd.read_csv(Location, sep=Separator)

df.columns = ["CASE_ID", "ACTIVITY",	"TIMESTAMP_IN",	"TIMESTAMP_OUT", "COLOR"] 
df_order = df.sort_values(['CASE_ID', 'TIMESTAMP_IN'])

print(df_order.head())
print(df_order.head(1))
print(df_order.tail(1))
print(df_order.tail(1))

   CASE_ID        ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR
4        1           Aalta  0.000000e+00   0.000000e+00    NaN
0        1        admision  1.614764e+08   1.614772e+08    NaN
1        1          triaje  1.614772e+08   1.614816e+08    NaN
2        1    rx solicitud  1.614816e+08   1.614831e+08    NaN
3        1  rx exploracion  1.614831e+08   0.000000e+00    NaN
   CASE_ID ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR
4        1    Aalta           0.0            0.0    NaN
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR
1742      937  admision  1.620759e+08            0.0    NaN
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR
1742      937  admision  1.620759e+08            0.0    NaN


margety tiempos

## prefilter

### thers elements with timestamp to 0, delete this elements

In [0]:
id_filter = df_order[df_order["TIMESTAMP_IN"] == 0]["CASE_ID"]
df_filter_out = df_order[~df_order["CASE_ID"].isin(id_filter)]
id_filter = df_order[df_order["TIMESTAMP_OUT"] == 0]["CASE_ID"]
df_filter_out = df_filter_out[~df_filter_out["CASE_ID"].isin(id_filter)]
df_filter_out = df_filter_out.sort_values(['CASE_ID', 'TIMESTAMP_IN'])

In [13]:
df_unorder = df_filter_out.sort_values("TIMESTAMP_IN").groupby("CASE_ID").tail(1)[["CASE_ID", "ACTIVITY"]] 
id_unorder = df_unorder[df_unorder["ACTIVITY"] != "Aalta"]["CASE_ID"]
df_filter_out = df_filter_out[~df_filter_out["CASE_ID"].isin(id_unorder)]
df_filter_out = df_filter_out.sort_values(['CASE_ID', 'TIMESTAMP_IN'])

print("Len before prefilter: "+str(len(df_order["CASE_ID"].drop_duplicates())))
print("Len after prefilter: "+str(len(df_filter_out["CASE_ID"].drop_duplicates())))

Len before prefilter: 807
Len after prefilter: 801


### check elements whose last element is not alta and first is not admision

In [10]:
(df_filter_out.sort_values("TIMESTAMP_IN").groupby("CASE_ID")["ACTIVITY"].first() ).drop_duplicates()

CASE_ID
2    admision
Name: ACTIVITY, dtype: object

## create routes frame

In [22]:
# creamos  las columnas
# join function => The join() method takes all items in an iterable and joins them into one string.
# reset _index -> Pandas reset_index() is a method to reset index of a Data Frame. reset_index() method sets a list of integer ranging from 0 to length of data as index.
df_routes = df_order.groupby('CASE_ID')['ACTIVITY'].agg(lambda x: ','.join(x)).reset_index()
df_routes['SIZE'] = df_routes['ACTIVITY'].apply(lambda x: len(x.split(',')))

df_routes.columns = ['CASE_ID', 'PATH', 'SIZE_PATH']
df_routes['NUM_ELEMENTS_SIZE'] = df_routes.groupby('SIZE_PATH')["CASE_ID"].transform(len)
df_routes['NUM_ELEMENTS_PATH'] = df_routes.groupby('PATH')["CASE_ID"].transform(len)

df_routes.head()


,CASE_ID,PATH,SIZE_PATH,NUM_ELEMENTS_SIZE,NUM_ELEMENTS_PATH
0,1,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
1,2,"admision,triaje,rx solicitud,Aalta",4,25,22
2,4,"admision,triaje,Aalta",3,12,10
3,5,"admision,triaje,rx solicitud,Aalta",4,25,22
4,6,"admision,triaje,rx solicitud,Aalta",4,25,22


In [24]:
print("min size path: "+str(min(df_routes["SIZE_PATH"])))
print("Max size path: "+str(max(df_routes["SIZE_PATH"])))
print("Mean size path: "+str(df_routes["SIZE_PATH"].mean()))
print("MEdian size path: "+str(df_routes["SIZE_PATH"].median()))
print("Num users per len route mean: "+str(df_routes["NUM_ELEMENTS_SIZE"].mean()))
print("Num users per len route median: "+str(df_routes["NUM_ELEMENTS_SIZE"].median()))

min size path: 1
Max size path: 6
Mean size path: 2.159851301115242
MEdian size path: 2.0
Num users per len route mean: 694.6158612143743
Num users per len route median: 748.0


In [25]:
df_routes.groupby("SIZE_PATH").size()

SIZE_PATH
1      1
2    748
3     12
4     25
5     16
6      5
dtype: int64

## and join data to create date and routes together

and join data beacuase we want also the size and number of elements

In [26]:
df_join = df.merge(df_routes).sort_values(["CASE_ID", "TIMESTAMP_IN"])
df_join = df_join[["CASE_ID", "ACTIVITY", "TIMESTAMP_IN", "TIMESTAMP_OUT", "PATH", "SIZE_PATH", "NUM_ELEMENTS_SIZE", "NUM_ELEMENTS_PATH"]]
df_join.head()

,CASE_ID,ACTIVITY,TIMESTAMP_IN,TIMESTAMP_OUT,PATH,SIZE_PATH,NUM_ELEMENTS_SIZE,NUM_ELEMENTS_PATH
4,1,Aalta,0.000000e+00,0.000000e+00,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
0,1,admision,1.614764e+08,1.614772e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
1,1,triaje,1.614772e+08,1.614816e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
2,1,rx solicitud,1.614816e+08,1.614831e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
3,1,rx exploracion,1.614831e+08,0.000000e+00,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1


process times

normalize times

In [28]:
df_timestamp = df_join.sort_values(['CASE_ID', 'TIMESTAMP_IN']).copy()
min_value =  df_timestamp["TIMESTAMP_IN"].min()-1
print(min_value)
df_timestamp["TIMESTAMP_IN"] = df_timestamp["TIMESTAMP_IN"]-min_value
df_timestamp["TIMESTAMP_OUT"] = df_timestamp["TIMESTAMP_OUT"]-min_value
df_timestamp.sort_values("TIMESTAMP_IN")
df_timestamp.head()

-1.0


,CASE_ID,ACTIVITY,TIMESTAMP_IN,TIMESTAMP_OUT,PATH,SIZE_PATH,NUM_ELEMENTS_SIZE,NUM_ELEMENTS_PATH
4,1,Aalta,1.000000e+00,1.000000e+00,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
0,1,admision,1.614764e+08,1.614772e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
1,1,triaje,1.614772e+08,1.614816e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
2,1,rx solicitud,1.614816e+08,1.614831e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1
3,1,rx exploracion,1.614831e+08,1.000000e+00,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1


In [29]:
df_timestamp["TIME_NODE"]= df_timestamp["TIMESTAMP_OUT"] - df_timestamp["TIMESTAMP_IN"]
df_timestamp.head(10)

,CASE_ID,ACTIVITY,TIMESTAMP_IN,TIMESTAMP_OUT,PATH,SIZE_PATH,NUM_ELEMENTS_SIZE,NUM_ELEMENTS_PATH,TIME_NODE
4,1,Aalta,1.000000e+00,1.000000e+00,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1,0.000000e+00
0,1,admision,1.614764e+08,1.614772e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1,8.395566e+02
1,1,triaje,1.614772e+08,1.614816e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1,4.409828e+03
2,1,rx solicitud,1.614816e+08,1.614831e+08,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1,1.452231e+03
3,1,rx exploracion,1.614831e+08,1.000000e+00,"Aalta,admision,triaje,rx solicitud,rx exploracion",5,16,1,-1.614831e+08
5,2,admision,1.614772e+08,1.614780e+08,"admision,triaje,rx solicitud,Aalta",4,25,22,7.693292e+02
6,2,triaje,1.614780e+08,1.614838e+08,"admision,triaje,rx solicitud,Aalta",4,25,22,5.769668e+03
7,2,rx solicitud,1.614838e+08,1.615233e+08,"admision,triaje,rx solicitud,Aalta",4,25,22,3.957154e+04
8,2,Aalta,1.615233e+08,1.615233e+08,"admision,triaje,rx solicitud,Aalta",4,25,22,0.000000e+00
9,4,admision,1.614780e+08,1.614786e+08,"admision,triaje,Aalta",3,12,10,5.737981e+02


# Informacioens

## De nodos time in noces

In [30]:
df_timestamp.groupby("ACTIVITY").mean()["TIME_NODE"]

ACTIVITY
Aalta             0.000000e+00
admision         -2.000947e+05
interconsulta    -1.078275e+07
respuesta         2.234568e+04
rx exploracion   -1.614831e+08
rx informe        1.248370e+04
rx solicitud      1.352128e+04
triaje           -2.668599e+06
Name: TIME_NODE, dtype: float64

## elementos en tamamaños de rutas

In [31]:
num_users = df_timestamp["CASE_ID"].drop_duplicates().count()
df_timestamp["PERCENTAGE_SIZE"]= df_timestamp["NUM_ELEMENTS_SIZE"]/num_users
df_timestamp[["SIZE_PATH", "NUM_ELEMENTS_SIZE", "PERCENTAGE_SIZE"]].drop_duplicates()

,SIZE_PATH,NUM_ELEMENTS_SIZE,PERCENTAGE_SIZE
4,5,16,0.019827
5,4,25,0.030979
9,3,12,0.014870
82,6,5,0.006196
231,2,748,0.926890
1742,1,1,0.001239


## elementos en rutas

In [32]:

df_timestamp["PERCENTAGE_PATH"]= df_timestamp["NUM_ELEMENTS_PATH"]/num_users
df_timestamp[["PATH", "SIZE_PATH", "NUM_ELEMENTS_PATH", "PERCENTAGE_PATH"]].drop_duplicates().sort_values("PERCENTAGE_PATH", ascending= False).head()

,PATH,SIZE_PATH,NUM_ELEMENTS_PATH,PERCENTAGE_PATH
231,"admision,Aalta",2,748,0.926890
5,"admision,triaje,rx solicitud,Aalta",4,22,0.027261
9,"admision,triaje,Aalta",3,10,0.012392
55,"admision,triaje,rx informe,rx solicitud,Aalta",5,6,0.007435
72,"admision,triaje,rx solicitud,interconsulta,Aalta",5,3,0.003717


In [33]:
df_timestamp[df_timestamp["SIZE_PATH"] == 6][["PATH", "SIZE_PATH", "NUM_ELEMENTS_PATH", "PERCENTAGE_PATH"]].drop_duplicates().sort_values("PERCENTAGE_PATH", ascending= False).head()


,PATH,SIZE_PATH,NUM_ELEMENTS_PATH,PERCENTAGE_PATH
82,"admision,triaje,interconsulta,respuesta,rx sol...",6,3,0.003717
130,"admision,triaje,interconsulta,rx solicitud,res...",6,1,0.001239
166,"admision,triaje,rx solicitud,interconsulta,res...",6,1,0.001239
